In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime, time

import modutils

In [2]:
def load_smnist_sample(fname):
    tmp0 = pd.read_csv(fname)
    if 'label' in tmp0:
        trgt = np.array(tmp0.label)
        full_mapping = list(enumerate(set(trgt)))
        fullmap2onehot = {j:i for (i,j) in full_mapping}
        trgt = np.array([fullmap2onehot[x] for x in trgt])
    else:
        trgt = None
        full_mapping = None
    srcf1d = np.array(tmp0[['pixel{0}'.format(j+1) for j in range(28*28)]]) / 255.0
    srcf2d = srcf1d.reshape(srcf1d.shape[0], 28, 28)
    return (srcf1d, srcf2d, trgt, full_mapping)

In [3]:
(dev_x1, dev_x2, dev_y, _) = load_smnist_sample('../DataSets/SignLanguage/sign_mnist_train.csv')
(train_x1, train_x2, train_y),(valid_x1, valid_x2, valid_y) = modutils.splitSample((dev_x1, dev_x2, dev_y), (0.7, 0.3))

In [9]:
#simple feed-forward nn with 100 units in 1 hidden layer (73% accuracy after 100 epoch and 4 sec per epoch))
tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_x = tf.placeholder(shape=(None, 28*28), dtype=tf.float32, name='x')
    tf_y = tf.placeholder(shape=(None,), dtype=tf.int32, name='y')
    tf_y1hot = tf.one_hot(tf_y, 24, name='y1hot')
    
tf_hid = tf.layers.dense(tf_x, 100, activation=tf.nn.elu)
tf_lgt = tf.layers.dense(tf_hid, 24)
tf_prob = tf.nn.softmax(tf_lgt, name='prob')
tf_pred = tf.reshape(tf.nn.top_k(tf_prob, name='pred')[1], (tf.shape(tf_prob)[0],))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_pred, tf_y), tf.float32))

tf_loss = tf.reduce_max(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y1hot, logits=tf_lgt))
tf_train = tf.train.AdamOptimizer(1e-3).minimize(tf_loss)

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())

In [21]:
#simple cnn (99.9% accuracy after 34 epochs and 66 sec per epoch and still improving!!!))
tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_x = tf.placeholder(shape=(None, 28, 28), dtype=tf.float32, name='x')
    tf_y = tf.placeholder(shape=(None,), dtype=tf.int32, name='y')
    tf_x0 = tf.reshape(tf_x, shape=(tf.shape(tf_x)[0], 28, 28, 1))
    tf_y1hot = tf.one_hot(tf_y, 24, name='y1hot')
    
tf_cp1 = tf.layers.max_pooling2d(tf.layers.conv2d(tf_x0, filters=10, kernel_size=3, padding='same'), pool_size=2, strides=2)
tf_cp2 = tf.layers.max_pooling2d(tf.layers.conv2d(tf_cp1, filters=5, kernel_size=3, padding='same'), pool_size=2, strides=2)
tf_cout = tf.reshape(tf_cp2, shape=(tf.shape(tf_x)[0], 7*7*5))
tf_lgt = tf.layers.dense(tf_cout, 24)
tf_prob = tf.nn.softmax(tf_lgt, name='prob')
tf_pred = tf.reshape(tf.nn.top_k(tf_prob, name='pred')[1], (tf.shape(tf_prob)[0],))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_pred, tf_y), tf.float32))

tf_loss = tf.reduce_max(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y1hot, logits=tf_lgt))
tf_train = tf.train.AdamOptimizer(1e-3).minimize(tf_loss)

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())

In [23]:
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
#tffw = tf.summary.FileWriter('D:/Jupyter/Logs/12RNN07-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 10
num_epochs = 100
checkpoints = 500

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
valid_batch = {tf_x: valid_x2, tf_y: valid_y}
train_tuple = (train_x2, train_y)
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        for (mini_x, mini_y) in modutils.shuffleBatches(train_tuple, batchSize=batch_size):
            train_batch = {tf_x:mini_x, tf_y:mini_y}
            
            #l0 = tfLoss.eval(feed_dict=train_batch)
            #t0 = time.perf_counter()
            for j in range(num_steps):
                tf_train.run(feed_dict=train_batch)
            #t1 = time.perf_counter()
            #l1 = tfLoss.eval(feed_dict=train_batch)
    
        te1 = time.perf_counter()
        [lv,ac] = tfs.run([tf_loss, tf_accuracy], feed_dict=valid_batch)
            #tffw.add_summary(summary, i)
            #if i%checkpoints == 0 and i > 0:
            #    p = tfsSaver.save(tfs, 'D:/Jupyter/mltest/Models-12RNN07/model-{0:04d}.ckpt'.format(i))
            #    print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac))

Epoch 0 (69.2 sec): 		VL:3.237		AC:0.117
Epoch 1 (69.3 sec): 		VL:3.195		AC:0.193
Epoch 2 (69.3 sec): 		VL:3.187		AC:0.299
Epoch 3 (67.6 sec): 		VL:3.164		AC:0.353
Epoch 4 (67.6 sec): 		VL:3.139		AC:0.482
Epoch 5 (67.7 sec): 		VL:3.152		AC:0.462
Epoch 6 (67.8 sec): 		VL:2.995		AC:0.588
Epoch 7 (67.7 sec): 		VL:3.010		AC:0.621
Epoch 8 (67.4 sec): 		VL:2.720		AC:0.647
Epoch 9 (67.9 sec): 		VL:2.691		AC:0.727
Epoch 10 (65.4 sec): 		VL:2.570		AC:0.758
Epoch 11 (66.0 sec): 		VL:2.231		AC:0.794
Epoch 12 (65.3 sec): 		VL:2.264		AC:0.832
Epoch 13 (65.3 sec): 		VL:2.169		AC:0.833
Epoch 14 (65.4 sec): 		VL:2.271		AC:0.838
Epoch 15 (65.4 sec): 		VL:2.041		AC:0.874
Epoch 16 (65.4 sec): 		VL:1.808		AC:0.912
Epoch 17 (66.1 sec): 		VL:2.085		AC:0.911
Epoch 18 (65.7 sec): 		VL:1.828		AC:0.927
Epoch 19 (65.4 sec): 		VL:2.514		AC:0.945
Epoch 20 (65.6 sec): 		VL:1.752		AC:0.969
Epoch 21 (65.5 sec): 		VL:1.904		AC:0.964
Epoch 22 (65.3 sec): 		VL:2.173		AC:0.974
Epoch 23 (65.6 sec): 		VL:1.705		AC:0.979
Ep

KeyboardInterrupt: 

In [31]:
#simple logistic regression (73% accuracy after 100 epoch and 2 sec per epoch)
tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_x = tf.placeholder(shape=(None, 28*28), dtype=tf.float32, name='x')
    tf_y = tf.placeholder(shape=(None,), dtype=tf.int32, name='y')
    tf_y1hot = tf.one_hot(tf_y, 24, name='y1hot')
    
tf_lgt = tf.layers.dense(tf_x, 24)
tf_prob = tf.nn.softmax(tf_lgt, name='prob')
tf_pred = tf.reshape(tf.nn.top_k(tf_prob, name='pred')[1], (tf.shape(tf_prob)[0],))
tf_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf_pred, tf_y), tf.float32))

tf_loss = tf.reduce_max(tf.nn.softmax_cross_entropy_with_logits(labels=tf_y1hot, logits=tf_lgt))
tf_train = tf.train.AdamOptimizer(1e-3).minimize(tf_loss)

tfsum_loss = tf.summary.scalar('CrossEntropy', tf_loss)
tfsum_accuracy = tf.summary.scalar('Accuracy', 1-tf_accuracy)
tfsum_all = tf.summary.merge([tfsum_loss, tfsum_accuracy])
tfsaver = tf.train.Saver(max_to_keep=2)

dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/17MNIST_01LOGREG-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 10
num_epochs = 1000
checkpoints = 500

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f}'
valid_batch = {tf_x: valid_x1, tf_y: valid_y}
train_tuple = (train_x1, train_y)
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        for (mini_x, mini_y) in modutils.shuffleBatches(train_tuple, batchSize=batch_size):
            train_batch = {tf_x:mini_x, tf_y:mini_y}
            
            for j in range(num_steps):
                tf_train.run(feed_dict=train_batch)
    
        te1 = time.perf_counter()
        [lv,ac,smr] = tfs.run([tf_loss, tf_accuracy, tfsum_all], feed_dict=valid_batch)
        tffw.add_summary(smr, i)
        
            #tffw.add_summary(summary, i)
            #if i%checkpoints == 0 and i > 0:
        p = tfsaver.save(tfs, 'D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-{0:04d}.ckpt'.format(i))
        print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac))

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0000.ckpt
Epoch 0 (2.29 sec): 		VL:4.016		AC:0.094
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0001.ckpt
Epoch 1 (2.67 sec): 		VL:4.346		AC:0.175
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0002.ckpt
Epoch 2 (2.17 sec): 		VL:3.814		AC:0.191
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0003.ckpt
Epoch 3 (2.66 sec): 		VL:3.977		AC:0.197
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0004.ckpt
Epoch 4 (2.54 sec): 		VL:3.753		AC:0.245
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0005.ckpt
Epoch 5 (2.34 sec): 		VL:3.558		AC:0.238
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0006.ckpt
Epoch 6 (2.21 sec): 		VL:3.683		AC:0.324
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0007.ckpt
Epoch 7 (2.21 sec): 		VL:3.711		A

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0065.ckpt
Epoch 65 (2.13 sec): 		VL:2.907		AC:0.637
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0066.ckpt
Epoch 66 (2.13 sec): 		VL:2.838		AC:0.689
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0067.ckpt
Epoch 67 (2.1 sec): 		VL:2.809		AC:0.694
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0068.ckpt
Epoch 68 (2.14 sec): 		VL:3.080		AC:0.637
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0069.ckpt
Epoch 69 (2.21 sec): 		VL:2.601		AC:0.661
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0070.ckpt
Epoch 70 (2.18 sec): 		VL:2.898		AC:0.704
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0071.ckpt
Epoch 71 (2.08 sec): 		VL:2.897		AC:0.582
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0072.ckpt
Epoch 72 (2.24 sec): 		VL:2

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0130.ckpt
Epoch 130 (2.23 sec): 		VL:2.501		AC:0.825
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0131.ckpt
Epoch 131 (2.24 sec): 		VL:2.342		AC:0.825
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0132.ckpt
Epoch 132 (2.47 sec): 		VL:2.472		AC:0.768
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0133.ckpt
Epoch 133 (2.37 sec): 		VL:2.152		AC:0.857
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0134.ckpt
Epoch 134 (2.06 sec): 		VL:2.662		AC:0.832
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0135.ckpt
Epoch 135 (2.4 sec): 		VL:2.304		AC:0.852
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0136.ckpt
Epoch 136 (2.37 sec): 		VL:2.389		AC:0.770
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0137.ckpt
Epoch 137 (2.06 sec)

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0195.ckpt
Epoch 195 (2.14 sec): 		VL:2.283		AC:0.922
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0196.ckpt
Epoch 196 (2.44 sec): 		VL:2.119		AC:0.866
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0197.ckpt
Epoch 197 (2.56 sec): 		VL:2.756		AC:0.863
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0198.ckpt
Epoch 198 (2.34 sec): 		VL:2.138		AC:0.919
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0199.ckpt
Epoch 199 (2.46 sec): 		VL:2.122		AC:0.928
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0200.ckpt
Epoch 200 (2.1 sec): 		VL:2.041		AC:0.938
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0201.ckpt
Epoch 201 (2.43 sec): 		VL:2.305		AC:0.907
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0202.ckpt
Epoch 202 (2.17 sec)

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0260.ckpt
Epoch 260 (2.15 sec): 		VL:1.991		AC:0.898
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0261.ckpt
Epoch 261 (2.11 sec): 		VL:1.879		AC:0.962
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0262.ckpt
Epoch 262 (2.11 sec): 		VL:1.981		AC:0.951
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0263.ckpt
Epoch 263 (2.4 sec): 		VL:1.896		AC:0.950
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0264.ckpt
Epoch 264 (2.32 sec): 		VL:2.518		AC:0.896
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0265.ckpt
Epoch 265 (2.65 sec): 		VL:1.890		AC:0.944
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0266.ckpt
Epoch 266 (2.34 sec): 		VL:2.001		AC:0.954
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0267.ckpt
Epoch 267 (2.21 sec)

...

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0715.ckpt
Epoch 715 (3.14 sec): 		VL:1.487		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0716.ckpt
Epoch 716 (2.96 sec): 		VL:1.625		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0717.ckpt
Epoch 717 (2.97 sec): 		VL:1.272		AC:0.998
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0718.ckpt
Epoch 718 (2.93 sec): 		VL:1.666		AC:0.998
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0719.ckpt
Epoch 719 (3.04 sec): 		VL:1.168		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0720.ckpt
Epoch 720 (2.97 sec): 		VL:1.407		AC:0.998
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0721.ckpt
Epoch 721 (2.87 sec): 		VL:1.661		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0722.ckpt
Epoch 722 (2.7 sec)

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0780.ckpt
Epoch 780 (2.77 sec): 		VL:1.133		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0781.ckpt
Epoch 781 (2.7 sec): 		VL:1.451		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0782.ckpt
Epoch 782 (2.73 sec): 		VL:1.712		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0783.ckpt
Epoch 783 (2.7 sec): 		VL:1.428		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0784.ckpt
Epoch 784 (2.66 sec): 		VL:1.200		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0785.ckpt
Epoch 785 (2.64 sec): 		VL:1.542		AC:0.995
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0786.ckpt
Epoch 786 (2.6 sec): 		VL:1.372		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0787.ckpt
Epoch 787 (2.6 sec): 	

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0845.ckpt
Epoch 845 (2.57 sec): 		VL:1.201		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0846.ckpt
Epoch 846 (2.39 sec): 		VL:1.484		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0847.ckpt
Epoch 847 (2.41 sec): 		VL:1.567		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0848.ckpt
Epoch 848 (2.52 sec): 		VL:1.863		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0849.ckpt
Epoch 849 (2.38 sec): 		VL:1.149		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0850.ckpt
Epoch 850 (2.4 sec): 		VL:1.202		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0851.ckpt
Epoch 851 (2.42 sec): 		VL:1.797		AC:0.998
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0852.ckpt
Epoch 852 (2.4 sec):

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0910.ckpt
Epoch 910 (2.68 sec): 		VL:1.036		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0911.ckpt
Epoch 911 (2.43 sec): 		VL:1.534		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0912.ckpt
Epoch 912 (2.43 sec): 		VL:1.433		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0913.ckpt
Epoch 913 (2.43 sec): 		VL:1.250		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0914.ckpt
Epoch 914 (2.45 sec): 		VL:1.306		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0915.ckpt
Epoch 915 (2.42 sec): 		VL:1.038		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0916.ckpt
Epoch 916 (2.42 sec): 		VL:1.658		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0917.ckpt
Epoch 917 (2.42 sec

Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0975.ckpt
Epoch 975 (2.47 sec): 		VL:1.307		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0976.ckpt
Epoch 976 (2.42 sec): 		VL:1.293		AC:0.999
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0977.ckpt
Epoch 977 (2.4 sec): 		VL:1.248		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0978.ckpt
Epoch 978 (2.41 sec): 		VL:1.375		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0979.ckpt
Epoch 979 (2.41 sec): 		VL:1.327		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0980.ckpt
Epoch 980 (2.42 sec): 		VL:1.056		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0981.ckpt
Epoch 981 (2.41 sec): 		VL:0.938		AC:1.000
Model saved at checkpoint: D:/Jupyter/mltest/Models-17SMNIST/model01LOGREG-0982.ckpt
Epoch 982 (2.44 sec)

### tmp = np.cumsum([1,2,3])

In [ ]:
train_y.shape, valid_y.shape

In [ ]:
z[(z > 0.3) & (z < 0.7),]

In [ ]:
z = np.random.uniform(size=dev_x1.shape[0])

In [ ]:
z.shape

In [ ]:
(q1, q2) = (dev_y[(z >= a) & (z < b),] for (a,b) in [(-0.1, 0.7),(0.7, 1.1)])

In [ ]:
q2.shape